# YOLOv8 model evaluation

In [1]:
from roboflow import Roboflow
import os

os.environ["DATASET_DIRECTORY"] = "C:/content/datasets"

rf = Roboflow(api_key="feG2z6aSS9JkkauzQ8Qo")
project = rf.workspace("lidar-object-detection").project("lidar-human-detection")
dataset = project.version(2).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.208, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to C:/content/datasets/lidar-human-detection-2 in yolov8:: 100%|██████████| 6458/6458 [00:02<00:00, 2259.86it/s]


In [2]:
import cv2
import numpy as np
from ouster import client
from ouster import pcap
from ultralytics import YOLO
from ouster.client import Scans, XYZLut, SensorInfo, destagger
from contextlib import closing
from collections import defaultdict


def avg_time_of_detection(weight_path):
    detection_time = []
    model = YOLO(weight_path)

    metadata_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.json"
    pcap_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.pcap"

    with open(metadata_path, 'r') as f:
        metadata = client.SensorInfo(f.read())

    pcap_file = pcap.Pcap(pcap_path, metadata)
    xyz_lut = client.XYZLut(metadata)

    with closing(Scans(pcap_file)) as scans:
        i = 0
        for scan in scans:
            i += 1
            sig_field = scan.field(client.ChanField.SIGNAL)
            sig_destaggered = destagger(metadata, sig_field)
            scaling_factor = 0.004
            scaled_arr = sig_destaggered / (0.5 + scaling_factor * sig_destaggered)
            signal_image = scaled_arr.astype(np.uint8)
            combined_img = np.dstack((signal_image, signal_image, signal_image))

            results = model.predict(source=combined_img, imgsz=1024, tracker='bytetrack.yaml', verbose=False)

            speed = results[0].speed
            time = speed['preprocess'] + speed['inference'] + speed['postprocess']
            detection_time.append(time)
            
            if i >= 100:   
                break
    return sum(detection_time)/len(detection_time)

In [9]:
from ultralytics import YOLO
import os


directory = '../weights/'

detection_results = {}
 
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        # print(f)
        avg_time = avg_time_of_detection(str(f))
        print(avg_time)
        model = YOLO(f)
        metrics = model.val()
        formatted_results = {}
        for key, val in metrics.results_dict.items():
            if key != 'fitness':
                formatted_results[key[8:-3]] = val
            else:
                formatted_results[key] = val
        formatted_results['mAP75'] = metrics.box.map75
        detection_results[f] = [avg_time, formatted_results]

118.77604722976685


Ultralytics YOLOv8.0.208  Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning C:\content\datasets\lidar-human-detection-1\valid\labels.cache... 269 images, 47 backgrounds, 0 corrupt: 100%|██████████| 269/269 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:32<00:00,  1.92s/it]
                   all        269        662      0.869      0.894      0.931      0.578
Speed: 0.3ms preprocess, 116.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\val


120.40929794311523


Ultralytics YOLOv8.0.208  Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning C:\content\datasets\lidar-human-detection-2\valid\labels.cache... 269 images, 47 backgrounds, 0 corrupt: 100%|██████████| 269/269 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:35<00:00,  2.11s/it]
                   all        269        662      0.884      0.883      0.935      0.583
Speed: 0.3ms preprocess, 128.7ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\val2


120.35186052322388


...

Ultralytics YOLOv8.0.208  Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning C:\content\datasets\lidar-human-detection-2\valid\labels.cache... 269 images, 47 backgrounds, 0 corrupt: 100%|██████████| 269/269 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:16<00:00,  1.06it/s]
                   all        269        662      0.891      0.869      0.933      0.582
Speed: 0.3ms preprocess, 55.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\val7


60.065436363220215


Ultralytics YOLOv8.0.208  Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning C:\content\datasets\lidar-human-detection-2\valid\labels.cache... 269 images, 47 backgrounds, 0 corrupt: 100%|██████████| 269/269 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:16<00:00,  1.04it/s]
                   all        269        662       0.86      0.902      0.933      0.575
Speed: 0.3ms preprocess, 56.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\val8


62.611849308013916


Ultralytics YOLOv8.0.208  Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning C:\content\datasets\lidar-human-detection-2\valid\labels.cache... 269 images, 47 backgrounds, 0 corrupt: 100%|██████████| 269/269 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:16<00:00,  1.05it/s]
                   all        269        662      0.857      0.893      0.921       0.56
Speed: 0.3ms preprocess, 55.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\val9


In [10]:
for key, val in detection_results.items():
    print(f"{key} :  {val[0]} ms, {val[1]}")

../weights/best_1000_m_200.pt :  118.77604722976685 ms, {'precision': 0.8692426144063043, 'recall': 0.893731455230514, 'mAP50': 0.9306214431611054, 'mAP50-95': 0.5784182247261054, 'fitness': 0.6136385465696055, 'mAP75': 0.6483697820971608}
../weights/best_3000_m_100.pt :  120.40929794311523 ms, {'precision': 0.8836347904546593, 'recall': 0.8832479082812953, 'mAP50': 0.9346218530879644, 'mAP50-95': 0.5825829980135697, 'fitness': 0.6177868835210092, 'mAP75': 0.6514991905842425}
../weights/best_3000_m_50.pt :  120.35186052322388 ms, {'precision': 0.9123084342595152, 'recall': 0.8580060422960725, 'mAP50': 0.930946122667509, 'mAP50-95': 0.5761372359219116, 'fitness': 0.6116181245964712, 'mAP75': 0.646474652402208}
../weights/best_3000_n_100.pt :  34.66836929321289 ms, {'precision': 0.8852701575502824, 'recall': 0.8655589123867069, 'mAP50': 0.9244299681155909, 'mAP50-95': 0.5504321634699545, 'fitness': 0.5878319439345181, 'mAP75': 0.5946786425634137}
../weights/best_3000_n_200.pt :  33.98355

## Benchmark

In [11]:
from ultralytics.utils.benchmarks import benchmark

benchmark(model='../weights/best_3000_s_100.pt', data='/content/datasets/lidar-human-detection-2/data.yaml', imgsz=1024, half=False)

Setup complete  (16 CPUs, 39.2 GB RAM, 398.6/1862.2 GB disk)

Benchmarks complete for best_3000_s_100.pt on /content/datasets/lidar-human-detection-2/data.yaml at imgsz=1024 (451.22s)
                   Format Status  Size (MB)  metrics/mAP50-95(B)  Inference time (ms/im)
0                 PyTorch              21.4               0.5824                   63.16
1             TorchScript              43.0               0.5823                  342.03
2                    ONNX               0.0                  NaN                     NaN
3                OpenVINO               0.0                  NaN                     NaN
4                TensorRT               0.0                  NaN                     NaN
5                  CoreML               0.0                  NaN                     NaN
6   TensorFlow SavedModel               0.0                  NaN                     NaN
7     TensorFlow GraphDef               0.0                  NaN                     NaN
8         Tenso

,Format,Status❔,Size (MB),metrics/mAP50-95(B),Inference time (ms/im)
0,PyTorch,✅,21.4,0.5824,63.16
1,TorchScript,✅,43.0,0.5823,342.03
2,ONNX,❌,0.0,NaN,NaN
3,OpenVINO,❌,0.0,NaN,NaN
4,TensorRT,❌,0.0,NaN,NaN
5,CoreML,❌,0.0,NaN,NaN
6,TensorFlow SavedModel,❌,0.0,NaN,NaN
7,TensorFlow GraphDef,❌,0.0,NaN,NaN
8,TensorFlow Lite,❌,0.0,NaN,NaN
9,TensorFlow Edge TPU,❌,0.0,NaN,NaN
